In [ ]:
!rm -rf ./*

# clone repositories

In [ ]:
!git clone https://github.com/obada-jaras/QA-Model.git
!git clone https://github.com/obada-jaras/Arabic-QA-Datasets.git
!git clone https://github.com/huggingface/transformers.git

In [ ]:
!git clone https://github.com/aub-mind/arabert
!mkdir arabert-model
!mv arabert arabert-model

!cp arabert-model/arabert/examples/question-answering/utils_qa.py arabert-model/
!cp arabert-model/arabert/examples/question-answering/trainer_qa.py arabert-model/
!cp arabert-model/arabert/examples/question-answering/run_qa.py arabert-model/
!cp arabert-model/arabert/examples/question-answering/squad_preprocessing.py arabert-model/

<hr>

# install requirements

In [ ]:
!pip install -r QA-Model/requirements.txt -q

<hr>

# combine datasets

In [ ]:
!python QA-Model/utils/merge_json_files.py --input "$(find 'Arabic-QA-Datasets/All Datasets' -name '*.json')" --output Arabic-QA-Datasets/merged_dataset.json

<hr>

# preprocess the data

In [ ]:
!mkdir Arabic-QA-Datasets/pre

model_name="aubmindlab/araelectra-base-discriminator"
!python arabert-model/squad_preprocessing.py \
  --input_file "Arabic-QA-Datasets/merged_dataset.json" \
  --output_file "Arabic-QA-Datasets/pre/merged_dataset_pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

<hr>

# split the data

In [ ]:
!python QA-Model/utils/data_split.py \
    --input_file "Arabic-QA-Datasets/pre/merged_dataset_pre.json" \
    --output_dir "Arabic-QA-Datasets/pre" \
    --train_ratio 0.8

<hr>

# create custome dataset loader

In [ ]:
!python QA-Model/utils/save_dataset_loader.py \
    --dataset_loader "QA-Model/utils/dataset_loader.py" \
    --train_file "../Arabic-QA-Datasets/pre/train_dataset.json" \
    --dev_file "../Arabic-QA-Datasets/pre/dev_dataset.json" \
    --output_file "arabert-model/dataset_loader.py"

<hr>

# train the model

In [ ]:
model_name="aubmindlab/araelectra-base-discriminator"

!python arabert-model/run_qa.py \
  --model_name_or_path $model_name \
  --dataset_name arabert-model/dataset_loader.py \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 5 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ./run \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 40000 \
  --seed 666 \
  --overwrite_output_dir \
  --warmup_steps 500

<hr>

# download the fine-tuned model

#### download the model directory to the local machine (from colab)

In [ ]:
# Import necessary libraries
import os
import shutil
from google.colab import files

# Define the directory you want to download
source_directory = "/content/run"

# Create a zip file from the directory
zip_file_name = "model.zip"
shutil.make_archive(zip_file_name.split('.')[0], 'zip', source_directory)

# Download the zip file
files.download(zip_file_name)

#### copy the model directory to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r "run" "/content/drive/MyDrive/fine-tunedAraBERT/"

<hr>

# use the model and test it

#### Load the trained model and tokenizer:

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "./run"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

#### Define a function to perform predictions using the loaded model and tokenizer:

In [ ]:
from transformers import pipeline

def answer_question(question, context):
    nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result = nlp(question=question, context=context)
    return result['answer']

#### Use the function to answer questions:

###### example #1

In [ ]:
context = "خلال قرابة قرن من الزمن، تحولت مدرسة بنات صغيرة في بلدة بيرزيت، إلى الجامعة الفلسطينية الأكثر تأثيرًا في مسيرة التعليم العالي الفلسطيني، وما رافق ذلك من تأثير على تشكيل الوعي والثقافة بمعناها المعرفي ومعناها المقاوم، الذي عُرفت به بيرزيت خلال تاريخها، فشكلت شوكة في حلق الاحتلال، عبر الإصرار على القيام بدورها التنويري، الذي ما زالت تقوم به بثبات، مشكلةً داخل حرمها مجتمعًا فلسطينيًّا متعدد الثقافات، قادرًا على الحوار والتواصل، ورافضًا للاحتلال كوصي على العقول والثقافة، على سبيل التحرر منه."

question1 = "ما الذي تحولت إليه مدرسة بنات بيرزيت؟"
answer1 = answer_question(question1, context)
print(answer1)

question2 = "ما العامل الرئيسي الذي أثر في تشكيل الوعي والثقافة في بيرزيت؟"
answer2 = answer_question(question2, context)
print(answer2)

question3 = "كيف تعاملت جامعة بيرزيت مع الاحتلال؟"
answer3 = answer_question(question3, context)
print(answer3)

question4 = "ما هو الدور الذي تقوم به جامعة بيرزيت بثبات؟"
answer4 = answer_question(question4, context)
print(answer4)

question5 = "ما هي الخصائص الرئيسية للمجتمع الفلسطيني داخل حرم جامعة بيرزيت؟"
answer5 = answer_question(question5, context)
print(answer5)

###### example #2

In [ ]:
context = "في إطار رسالتھا العامة، تولي جامعة بيرزيت عناية خاصة للبحث العلمي وتعتبره من أھم وظائفها وتنظر إليه كسبيل رئيسي لرفع مستواھا العلمي وتمكينھا من المنافسة وخدمة المجتمع الفلسطيني بشكل خاص، والمجتمع العربي بشكل عام، ولزيادة المعرفة الإنسانية. كما ترى الجامعة في البحث العلمي جزءا ھاما من عمل أعضاء ھيئة التدريس فيھا، وتراعي ھذا المبدأ عند تعيين أعضاء ھيئة التدريس وتوزيع أعبائھم ، وتحويل وضعھم الوظيفي من غير منتظم إلى منتظم . كما تسعى الجامعة إلى منح طلبتھا، من كافة المستويات، فرصة الحصول على مھارات البحث العلمي والمشاركة فيه. كما تلتزم جامعة بيرزيت بالسعي لتوفير البيئة الملائمة والإمكانيات اللازمة للنھوض بالبحث العلمي، من حيث توفير الوقت لأعضاء ھيئة التدريس للقيام بالبحث وتوفير الأماكن والأجھزة والخدمات المكتبية والحاسوبية والدعم المالي وأية متطلبات أخرى يحتاجھا البحث العلمي."

question1 = "ما الغرض الرئيسي لتولي جامعة بيرزيت عناية خاصة بالبحث العلمي؟"
answer1 = answer_question(question1, context)
print(answer1)

question2 = "كيف تعتبر جامعة بيرزيت البحث العلمي جزءاً من عمل أعضاء هيئة التدريس؟"
answer2 = answer_question(question2, context)
print(answer2)

question3 = "ما الذي تسعى الجامعة إلى منح طلابها؟"
answer3 = answer_question(question3, context)
print(answer3)

question4 = "كيف تلتزم جامعة بيرزيت بتوفير البيئة الملائمة والإمكانيات اللازمة للنهوض بالبحث العلمي؟"
answer4 = answer_question(question4, context)
print(answer4)

question5 = "ما هي المتطلبات التي يحتاجها البحث العلمي حسب جامعة بيرزيت؟"
answer5 = answer_question(question5, context)
print(answer5)

question6 = "لمن تقدم جامعة بيرزيت خدماتها بشكل خاص؟"
answer6 = answer_question(question6, context)
print(answer6)

question7 = "كيف تنظر جامعة بيرزيت إلى البحث العلمي؟"
answer7 = answer_question(question7, context)
print(answer7)

question8 = "ماذا تراعي جامعة بيرزيت عند تعيين أعضاء هيئة التدريس؟"
answer8 = answer_question(question8, context)
print(answer8)

question9 = "ما الهدف من تحويل وضع أعضاء هيئة التدريس من غير منتظم إلى منتظم؟"
answer9 = answer_question(question9, context)
print(answer9)

question10 = "ما هي المستويات التي تشملها جامعة بيرزيت في منح طلابها مهارات البحث العلمي؟"
answer10 = answer_question(question10, context)
print(answer10)

###### example #3

In [ ]:
context = "كان الشاعر نزار قباني شاعرًا سوريًا معروفًا. وُلِد في دمشق في الثالث من مارس 1923 وتوفي في لندن في 30 أبريل 1998."

question1 = "متى ولد نزار قباني؟"
answer1 = answer_question(question1, context)
print(answer1)

question2 = "في أي مدينة وُلد نزار قباني؟"
answer2 = answer_question(question2, context)
print(answer2)

question3 = "متى توفي نزار قباني؟"
answer3 = answer_question(question3, context)
print(answer3)

question4 = "في أي مدينة توفي نزار قباني؟"
answer4 = answer_question(question4, context)
print(answer4)

question5 = "ما الجنسية التي ينتمي إليها الشاعر نزار قباني؟"
answer5 = answer_question(question5, context)
print(answer5)

###### example #4

In [ ]:
context = "كان الشاعر نزار قباني شاعرًا سوريًا معروفًا. وُلِد في دمشق في الثالث من مارس 1923 وتوفي في لندن في 30 أبريل 1998."

question = "متى ولد صلاح الدين الايوبي؟"
answer = answer_question(question, context)
print(answer)